In [ ]:
# Install the libraries
!pip install PyPortfolioOpt

In [ ]:
# Import the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from pypfopt.cla import CLA
from pypfopt import plotting
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientFrontier

In [ ]:
# Load the data
from google.colab import files
uploaded = files.upload() # Coindesk

In [ ]:
# Store the data into dataframes
bitcoin = pd.read_csv("BTC_USD_2019-04-26_2020-04-25-CoinDesk.csv")   # Bitcoin
ethereum = pd.read_csv("ETH_USD_2019-04-26_2020-04-25-CoinDesk.csv")  # Ethereum
litecoin = pd.read_csv("LTC_USD_2019-04-26_2020-04-25-CoinDesk.csv")  # Litecoin
neo = pd.read_csv("NEO_USD_2019-04-26_2020-04-25-CoinDesk.csv")       # Neo
monero = pd.read_csv("XMR_USD_2019-04-26_2020-04-25-CoinDesk.csv")    # Monero
tron = pd.read_csv("TRX_USD_2019-04-26_2020-04-25-CoinDesk.csv")      # Tron
dash = pd.read_csv("DASH_USD_2019-04-26_2020-04-25-CoinDesk.csv")     # Dash     

FileNotFoundError: ignored

In [ ]:
# Print the data
print(bitcoin)
print(ethereum)
print(litecoin)
print(neo)
print(monero)
print(tron)
print(dash)

In [ ]:
# Show the first 5 rows of data
bitcoin.head()
ethereum.head()
litecoin.head()
neo.head()
monero.head()
tron.head()
dash.head()

In [ ]:
# Create a new dataframe that holds the closing price of all 7 cryptocurrencies
df = pd.DataFrame({"Date": bitcoin["Date"],
                   "Bitcoin": bitcoin["Closing Price (USD)"],
                   "Ethereum": ethereum["Closing Price (USD)"],
                   "Litecoin": litecoin["Closing Price (USD)"],
                   "Neo": neo["Closing Price (USD)"],
                   "Monero": monero["Closing Price (USD)"],
                   "Tron": tron["Closing Price (USD)"],
                   "Dash": dash["Closing Price (USD)"]})

In [ ]:
# Get the df
df

In [ ]:
# Replace the index with date in the dataframe
df["Date"] = pd.to_datetime(df["Date"])
df.set_index('Date', inplace = True)
df

In [ ]:
# Assign weights to the cryptocurrency.
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
# Visually show the portfolio of cryptocurrency

# Create and plot the graph
for c in df.columns.values:
  plt.plot(df[c], label = c)

plt.title("Portfolio of Closing Price History")
plt.xlabel("Date", fontsize = 12)
plt.ylabel("Closeing Price in USD", fontsize = 12)
plt.legend(df.columns.values, loc = "upper left", fontsize = 5)
plt.show()

In [ ]:
# To take a better look at the graph
# Scale the data
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(feature_range = (0, 100))
scaled = min_max_scaler.fit_transform(df)
scaled

# Convert the scaled data into a dataframe
df_scaled = pd.DataFrame(scaled, columns = df.columns)

# Visualize the scaled data
my_crypto = df_scaled
plt.figure(figsize = (16, 10))

for c in my_crypto.columns.values:
  plt.plot(my_crypto[c], label = c)

plt.title("Cryptocurrency Scaled Graph")
plt.xlabel("Days")
plt.ylabel("Crypto Scaled Price ($)")
plt.legend(my_crypto.columns.values, loc = "upper left")
plt.show()

In [ ]:
# Show the daily simple return
returns = df.pct_change()
returns

In [ ]:
# Visualize the daily simple return
plt.figure(figsize = (16, 7))

for c in returns.columns.values:
  plt.plot(returns.index, returns[c], label = c, lw = 2, alpha = 0.7)

plt.title("Daily Simple Return")
plt.ylabel("Percentage (in decimal form)")
plt.xlabel("Days")
plt.legend(returns.columns.values, loc = "upper right")
plt.show()

In [ ]:
# Get the Volatility
print("The cryptocurrency volatility: ")
returns.std()

In [ ]:
# Show the mean/average of simple daily return
returns.mean()

In [ ]:
# Show the sum of simple daily return
returns.sum()

In [ ]:
# Get the correlation of simple daily return
returns.corr()

In [ ]:
# Visualize the correlation
plt.subplots(figsize = (14, 8))
sns.heatmap(returns.corr(), annot = True, fmt = "0.2%")

In [ ]:
# Get the daily cumulative simple returns
cum_returns = (returns + 1).cumprod()
cum_returns

In [ ]:
# Visualize the daily commulative simple return
plt.figure(figsize = (16, 8))

for c in cum_returns.columns.values:
  plt.plot(cum_returns.index, cum_returns[c], lw = 2, label = 2)

plt.title("Daily Cummulative Simple Return")
plt.xlabel("Days")
plt.ylabel("Growth of 1$ Investment")
plt.legend(cum_returns.columns.values, loc = "upper left", fontsize = 10)
plt.show()

In [ ]:
# Create and show the annualized covariance matrix
cov_matrix_annual = returns.cov() * 365
cov_matrix_annual

In [ ]:
# Calculate the portfolio variance
port_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
port_variance

In [ ]:
# Calculate the portfolio volatility (std)
port_volatility = np.sqrt(port_variance)
port_volatility

In [ ]:
# Calculate the annual portfolio return
portfolio_simple_annual_return = np.sum(returns.mean() * weights) * 365
portfolio_simple_annual_return

In [ ]:
# Show the expected annual return, volatility(risk) and variance

percent_var = str(round(port_variance, 2) * 100) + "%"
percent_volt = str(round(port_volatility, 2) * 100) + "%"
percent_retn = str(round(portfolio_simple_annual_return, 2) * 100) + "%"

print("Expected annual return: " + percent_retn)
print("Annual volatility/risk: " + percent_volt)
print("Anuual variance: " + percent_var)

In [ ]:
# Portfolio Optimization

# Calculate expected return and anuualized sample covariance matrix of asset returns
mu = expected_returns.mean_historical_return(df) 
s = risk_models.sample_cov(df)

# Optimize for max Sharpe ratio
ef = EfficientFrontier(mu, s)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose = True)

In [ ]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = 100000)

allocation, leftover = da.lp_portfolio()
print("Discrete allocation: ", allocation)
print("Funds reamining: ${:.2f}".format(leftover))

In [ ]:
# Plotting the waeight distribution
Plotting.plot_weights(weights)

In [ ]:
# Plot of covearience matrix annual
Plotting.plot_covariance(cov_matrix_annual)

In [ ]:
# Ploting the efficient frontier
efficient_portfolio = CLA(mu, s)
efficient_portfolio.max_sharpe()
efficient_portfolio.plot_efficient_frontier()